In [11]:
using LsqFit
using PyPlot

include("../compute_ttv.jl")
using TTVFaster

## 2 planets

In [2]:
p1 = readdlm("../ttv_planet1.txt");
p2 = readdlm("../ttv_planet2.txt");

In [3]:
data = [p1;p2][:]
function model_func(useless_input_x, params) #curve fit makes us take the first parameter
    
    planet1 = Planet_plane_hk(params[1],params[2],params[3],params[4],params[5]);
    planet2 = Planet_plane_hk(params[6],params[7],params[8],params[9],params[10]);

    ttv1 = zeros(length(p1));
    ttv2 = zeros(length(p2));
    
    ttv_data_p1 = planet1.period.*collect(1:1:length(p1)) + planet1.trans0
    ttv_data_p2 = planet2.period.*collect(1:1:length(p2)) + planet2.trans0

    # compute_ttv!(5, planet1, planet2, p1[:], p2[:], ttv1, ttv2);
    compute_ttv!(25, planet1, planet2, ttv_data_p1, ttv_data_p2, ttv1, ttv2);

    results1 = zeros(length(p1))
    results2 = zeros(length(p2))
    for j=1:length(p1)
        results1[j] = results1[j] + (j-1)*planet1.period + planet1.trans0
    end

    for j=1:length(p2)
        results2[j] = results2[j] + (j-1)*planet2.period + planet2.trans0
    end

    results2 = results2 + ttv2;
    results1 = results1 + ttv1;

    results = [results1;results2]
    return results
end;

In [4]:
p0 = [2.44e-6, 224.7008, 8445, 0.5, 0.5, 3e-6, 365.256, 8461, 0.5, 0.5]
fit = curve_fit(model_func, [], data, p0);

# curve fit returns:
- mass_ratio
- period
- trans0
- ecosw
- esinw

note: e times cos or sin of longitude of periastron measured from line of sight, in radians:

In [5]:
fit.param

10-element Array{Float64,1}:
   -3.77575e-8
  224.701     
 8445.5       
   -9.9815    
   -0.130602  
   -1.17061e-7
  365.256     
 8461.76      
   -7.60067   
    0.344813  

In [14]:
weight = 1./0.00001^2
nparam = length(fit.param)
println("Best-fit parameters: ",fit.param)


pname = ["mass ratio","period (d)", "t0 (d)","ecosw (d)", "esinw", 
    "mass ratio", "period (d)", "t0 (d)","ecosw (d)","esinw"]

println("Chi-square: ",chi_best)
period = fit.param[2]


Best-fit parameters: [-3.77574851073471e-8,224.70076795666418,8445.499590841526,-9.981500967638935,-0.13060234272648255,-1.1706098250151344e-7,365.2563663180399,8461.759145231861,-7.600671158033065,0.34481296815456314]


LoadError: LoadError: UndefVarError: chi_best not defined
while loading In[14], in expression starting on line 9

In [12]:

# Now, run an affine-invariant markov chain:
# Foreman-Mackey et al. (2014) - 'emc' 'mcmc hammer'

nparam = length(fit.param)

nwalkers = nparam * 3
nsteps = 10000
#nsteps = 100
# Set up arrays to hold the results:
par_mcmc = zeros(nwalkers,nsteps,nparam)
chi_mcmc = zeros(nwalkers,nsteps)
# Initialize walkers:
par_trial = fit.param
for j=1:nwalkers
# Select from within uncertainties:
  chi_trial = 1e100
    chi_best = 10
# Only initiate models with reasonable chi-square values:
  while chi_trial > (chi_best + 1000)
    par_trial = fit.param + errors.*randn(nparam)
    model =transit_model(time,par_trial)
    chi_trial = sum(((flux-model)./eflx).^2)
  end
  chi_mcmc[j,1]=chi_trial
  par_mcmc[j,1,:]=par_trial
  println("Success: ",par_trial,chi_trial)
end
# Initialize scale length & acceptance counter:
ascale = 2.0
accept = 0
# Next, loop over steps in markov chain:
for i=2:nsteps
  for j=1:nwalkers
    ipartner = j
# Choose another walker to 'breed' a trial step with:
    while ipartner == j
      ipartner = ceil(Int,rand()*nwalkers)
    end
# Now, choose a new set of parameters for the trial step:
    z=(rand()*(sqrt(ascale)-1.0/sqrt(ascale))+1.0/sqrt(ascale))^2
    par_trial=vec(z*par_mcmc[j,i-1,:]+(1.0-z)*par_mcmc[ipartner,i-1,:])
# Compute model & chi-square:    
    model_trial =transit_model(time,par_trial)
    chi_trial=sum(((flux-model_trial)./eflx).^2)
# Next, determine whether to accept this trial step:
    alp = z^(nparam-1)*exp(-0.5*(chi_trial - chi_mcmc[j,i-1]))
    if alp >= rand()
# If step is accepted, add it to the chains!
      par_mcmc[j,i,:] = par_trial
      chi_mcmc[j,i,:] = chi_trial
      accept = accept + 1
    else
# If step is rejected, then copy last step:
      par_mcmc[j,i,:] = par_mcmc[j,i-1,:]
      chi_mcmc[j,i,:] = chi_mcmc[j,i-1]
    end
  end
  if mod(i,1000) == 0
    frac_acc = accept/(1000*nwalkers)
    println("Number of steps: ",i," acceptance rate: ",frac_acc)
    ascale = 1.0 + (frac_acc/0.25)*(ascale-1.0)
    accept = 0
  end
end

# Now, determine time of burn-in by calculating first time median is crossed:
iburn = 0
for i=1:nparam
  med_param=median(par_mcmc[1:nwalkers,1:nsteps,i])
  for j=1:nwalkers
    istep=2
    while (par_mcmc[j,istep,i] > med_param) == (par_mcmc[j,istep-1,i] > med_param) && (istep < nsteps)
      istep=istep+1
    end
    if istep >= iburn
      iburn = istep
    end
  end
end

println("Burn-in ends: ",iburn)

# Also need trace plots & compute autocorrelation length [ ]

# Compute the various parameters:

pavg = mean(vec(par_mcmc[1:nwalkers,iburn:nsteps,1]));
psig = std(vec(par_mcmc[1:nwalkers,iburn:nsteps,1]));
@printf "Period: %8.4f +/- %8.4f\n" pavg psig

b=vec(par_mcmc[1:nwalkers,iburn:nsteps,4]);
bavg = mean(b)
bsig = std(b)
@printf "Impact par: %5.2f +/- %5.2f\n" bavg bsig

davg = mean(vec(par_mcmc[1:nwalkers,iburn:nsteps,5].^2*100));
dsig = std(vec(par_mcmc[1:nwalkers,iburn:nsteps,5].^2*100));
@printf "Depth (pct):   %5.2f +/- %5.2f\n" davg dsig

tdur = vec(par_mcmc[1:nwalkers,iburn:nsteps,3]);
tavg = mean(tdur*24)
tsig = std(tdur*24)
@printf "Duration (hr):   %5.2f +/- %5.2f\n" tavg tsig

using CGS
rho = 3/pi^2./(tdur.*24*3600).^3.*(1-b.^2).*(period.*24*3600)/GRAV;
#rho = 3/pi^2./(tdur.*24*3600).^3.*(period.*24*3600)/GRAV;
ravg = mean(rho)
rsig = std(rho)
@printf "Stellar density (g/cc):   %5.2f +/- %5.2f\n" ravg rsig

for i=1:nparam
  for j=1:nwalkers
    plot(vec(par_mcmc[j,1:nsteps,i]))
  end
  xlabel("MCMC step")
  ylabel(pname[i])
  println("Hit return to continue")
  read(STDIN,Char)
  clf()
end

for i=2:nparam
  for j=1:i-1
    scatter(vec(par_mcmc[1:nwalkers,iburn:nsteps,i]),vec(par_mcmc[1:nwalkers,iburn:nsteps,j]))
    xlabel(pname[i])
    ylabel(pname[j])
    println("Hit return to continue")
    read(STDIN,Char)
    clf()
  end
end

LoadError: LoadError: UndefVarError: errors not defined
while loading In[12], in expression starting on line 15